In [1]:
import nltk
import numpy as np
from scipy.io import loadmat
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import Lasso, Ridge
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score
from scipy import stats
from sklearn.decomposition import PCA
from IPython.core.debugger import set_trace
import pickle
import os
from tqdm import tqdm
import random
from joblib import Parallel, delayed

random.seed(97)
np.random.seed(97)

In [2]:
def compute_sig(sub,feat):
    print("Subject {} Feature {}".format(sub,feat))
        
    save_dir = root_save_dir + feat + "/"
    if os.path.exists(root_save_dir + feat + "/{}_sig.npy".format(sub)):
        return 0.0002
    
    reals = []
    for split_num in range(4):
        reals.append(np.load(save_dir + sub + "_y_test_{}.npy".format(split_num)))
    all_reals = np.vstack(reals)

    preds = []
    for split_num in range(4):
        preds.append(np.load(save_dir + sub + "_y_pred_{}.npy".format(split_num)))
    all_preds = np.vstack(preds)

    real_r2 = np.load(save_dir + sub + "_r2s.npy")
    greater = np.zeros(preds[0].shape[1])

    blocks = []
    for i in range(4):
        p = np.array([preds[i][j:j+blocksize] for j in range(0,len(preds[i]),blocksize)])
        blocks.append(p)

    for s in tqdm(range(iters)):
        new_preds = []
        for i in range(4):
            perm = np.random.permutation(len(blocks[i]))
            bl = blocks[i][perm]
            shuffled_preds = np.vstack(bl)
            new_preds.append(shuffled_preds)
        new_preds = np.vstack(new_preds)
        shuffled_r2 = r2_score(all_reals,new_preds,multioutput="raw_values")
        shuffled_r2[real_r2 == 0] = 0
        greater[shuffled_r2 >= real_r2] += 1            
    sig = greater/iters
    sig[sig == 0] = 1/iters
    np.save(root_save_dir + feat + "/{}_sig.npy".format(sub),sig)
    return np.min(sig)

In [3]:
root_save_dir = "predictions_remove_overlap/"
if not os.path.exists(root_save_dir):
    os.mkdir(root_save_dir)

all_features = ["punct_final"]

print(all_features)
print(len(all_features))
all_subjects = ["F","G","H","I","J","K","L","M","N"]
sub_data = "sub_space_data/"

blocksize = 10
iters = 5000

Parallel(n_jobs=28)(delayed(compute_sig)(sub,feat) for sub in all_subjects for feat in all_features)

['punct_final']
1


[0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002]